In [1]:
from os import listdir
from os.path import isfile, join
import os.path
import openpyxl
import pandas as pd
import math
import openpyxl
import pickle
import time

In [2]:
#Definiciones iniciales.
#Defininos los ficheros pickle para no perder procesamientos de ficheros ante posibles caidas del notebook
dffileNodosEmpresa = 'NodosEmpresa'
dffileRelacionesCargos = 'RelacionesCargos'
dffileRelacionesParticipadas ='RelacionesEmpresasParticipadas'
counter_file_name = 'file_processed_counter'

#Definimos los paths donde se encuentran los ficheros de SABI
path1 = '../../data/datasets/SABI/lista1'
path2 = '../../data/datasets/SABI/lista2'
onlyfiles1 = [f for f in listdir(path1) if isfile(join(path1, f))]
onlyfiles2 = [f for f in listdir(path2) if isfile(join(path2, f))]

#Cargamos en un vector, la lista de los ficheros que se van procesando, por si se cae el jupyter, no volver a procesar todos
#Sino proseguir con aquellos que no se han procesado.
if (not(os.path.isfile(counter_file_name))):
    with open(counter_file_name, 'wb') as dumpPickleCounter:
        pickle.dump(onlyfiles1, dumpPickleCounter)
else:
    with open(counter_file_name, 'rb') as dumpPickleCounter:
        onlyfiles1 = pickle.load(dumpPickleCounter)

print onlyfiles1

['42 Lista1 Empresas 233702_239401 (1).xlsx', '39 Lista1 Empresas 216602_222301.xlsx', '44 Lista1 Empresas 245102_250801.xlsx', '21 Lista1 Empresas 114002-119701.xlsx', '18 Lista1 Empresas 96902-102601.xlsx', '26 Lista1 Empresas 142502-148201.xlsx', '05 Lista1 Empresas 22802-28501.xlsx', '37 Lista1 Empresas 205202_210901.xlsx', '31 Lista1 Empresas 171002_176701.xlsx', '51 Lista1 Empresas 279001_280115.xlsx', '47 Lista1 Empresas 259001_264000.xlsx', '13 Lista1 Empresas 68401-74101.xlsx', '20 Lista1 Empresas 108301-114001.xlsx', '22 Lista 1 Empresas 119701-125401.xlsx', '30 Lista1 Empresas 165301-171001.xlsx', '16 Lista1 Empresas 85502-91201.xlsx', '33 Lista1 Empresas 182402-188101 (1).xlsx', '02 Lista1 Empresas 5701-11400.xlsx', '17 Lista1 Empresas 91201-96901.xlsx', '06 Lista1 Empresas 28502-34201.xlsx', '43 Lista1 Empresas 239402_245101.xlsx', '11 Lista1 Empresas 57002-62701.xlsx']


In [3]:
#Datos de prueba
# path = './testdata'
# onlyfiles = [f for f in listdir(path) if isfile(join(path, f))]
# print onlyfiles
#f = '../../data/datasets/SABI/01 Lista1 Empresas 1-5700.xlsx'
#f = 'SABITest.xlsx'

Definimos una funcion a la que se le pase un fichero, y añada a un dataframe los registros que necesitamos. Ojo a los campos que se definen como útiles de la excel, en cuyo caso si queremos añadir mas, solo tenemos que completar esta funcion. 
Se crean 2 dataframes, uno con los datos de las empresas y otro con las relaciones entre empresas y participadas. 

In [4]:
#Proceso que genera un DF con los nodos empresa y sus datos 
#un dataframe con las relaciones de las empresas participadas.
#un dataframe con los cargos. 
#Todo sacado de los ficheros de la lista1
#Chequeamos si los DF existen inicialmente en la carpeta. Esto es para evitar que si se cae el Jupyter comenzar de 0

if (not(os.path.isfile(dffileRelacionesParticipadas))):
    dfrelacionesParticipadas = pd.DataFrame(columns=('nombre', 'CIF', 'tipoRelacion','nombre','participadaId','participadaCodPais','porcentajeDirecto','porcentajeTotal'))
else:
    print 'loading dataframe dfrelacionesParticipadas from pickle file......'
    dfrelacionesParticipadas = pd.read_pickle(dffileRelacionesParticipadas)

if (not(os.path.isfile(dffileRelacionesCargos))):
    dfrelacionesCargos = pd.DataFrame(columns=('nombre', 'CIF', 'tipoRelacion','nombre','fechaNombramiento','actual'))
else:
    print 'loading dataframe dfrelacionesCargos from pickle file......'
    dfrelacionesCargos =pd.read_pickle(dffileRelacionesCargos)

if (not(os.path.isfile(dffileNodosEmpresa))):   
    dfwork = pd.DataFrame(columns=('nombre','CIF','ISIN','forma','fconst','Estado','fechaCambioEstado','Pres','DE','tipoCuentasDisponible','numEmpresasEnCorporacion','numparticipadas','codPrimarioCNAE','incidenciasJudiciales','reclamacionesAdministrativas','numMiembrosJuntas','numMiembrosJuntasDireccion','numEmpresasGrupo','long','lat','cotizoBolsa','totActivo2015','totActivo2014','fondosPropios2015','fondosPropios2014','numEmpleados2015','numEmpleados2015','netoVentas2015','netoVentas2014','file'))
else:
    print 'loading dataframe dfwork from pickle file......'
    dfwork = pd.read_pickle(dffileNodosEmpresa)
    
def append_file_to_df(f):
    wb = openpyxl.load_workbook(f)
    sheetnames = wb.get_sheet_names()
    sheet =  wb.get_sheet_by_name('Resultados')
    lastname = ''
    lastcif=''
    for rowNum in range(3, sheet.max_row):  # skip the first row
        if not(pd.isnull(sheet.cell(row=rowNum, column=2).value)):
            lastname=sheet.cell(row=rowNum, column=2).value
            lastcif=sheet.cell(row=rowNum, column=3).value
            dfwork.loc[len(dfwork)] = [sheet.cell(row=rowNum, column=2).value,sheet.cell(row=rowNum, column=3).value,sheet.cell(row=rowNum, column=6).value,sheet.cell(row=rowNum, column=8).value,sheet.cell(row=rowNum, column=9).value,sheet.cell(row=rowNum, column=10).value,sheet.cell(row=rowNum, column=11).value,sheet.cell(row=rowNum, column=12).value,sheet.cell(row=rowNum, column=13).value,sheet.cell(row=rowNum, column=15).value,sheet.cell(row=rowNum, column=16).value,sheet.cell(row=rowNum, column=17).value,sheet.cell(row=rowNum, column=18).value,sheet.cell(row=rowNum, column=19).value,sheet.cell(row=rowNum, column=22).value,sheet.cell(row=rowNum, column=27).value,sheet.cell(row=rowNum, column=28).value,sheet.cell(row=rowNum, column=41).value,sheet.cell(row=rowNum, column=55).value,sheet.cell(row=rowNum, column=56).value,sheet.cell(row=rowNum, column=57).value,sheet.cell(row=rowNum, column=58).value,sheet.cell(row=rowNum, column=59).value,sheet.cell(row=rowNum, column=60).value,sheet.cell(row=rowNum, column=61).value,sheet.cell(row=rowNum, column=62).value,sheet.cell(row=rowNum, column=63).value,sheet.cell(row=rowNum, column=64).value,sheet.cell(row=rowNum, column=65).value,f]
        if pd.isnull(sheet.cell(row=rowNum, column=2).value):
            if not(pd.isnull(sheet.cell(row=rowNum, column=49).value)):
                dfrelacionesParticipadas.loc[len(dfrelacionesParticipadas)] = [lastname,lastcif,'participada',sheet.cell(row=rowNum, column=49).value,sheet.cell(row=rowNum, column=50).value,sheet.cell(row=rowNum, column=52).value,sheet.cell(row=rowNum, column=53).value,sheet.cell(row=rowNum, column=54).value]
        if not(pd.isnull(sheet.cell(row=rowNum, column=29).value)) and not(pd.isnull(sheet.cell(row=rowNum, column=32).value)):
            dfrelacionesCargos.loc[len(dfrelacionesCargos)] = [lastname,lastcif,sheet.cell(row=rowNum, column=32).value,sheet.cell(row=rowNum, column=29).value,sheet.cell(row=rowNum, column=33).value,sheet.cell(row=rowNum, column=35).value]

loading dataframe dfrelacionesParticipadas from pickle file......
loading dataframe dfrelacionesCargos from pickle file......
loading dataframe dfwork from pickle file......


Para cada fichero dentro de la ruta, procesamos cada fichero añadiendo al dataframe los registros.

In [ ]:
#Primero procesamos los datos base de la lista1, creando los nodos empresa y la lista de participadas.
iterator = list(onlyfiles1)
for file in iterator:
    f=join(path1, file)
    print (time.strftime("%H:%M:%S")),' processing file: ',f
    append_file_to_df(f)
    print (time.strftime("%H:%M:%S")),'file: ',f,' appended'
    dfrelacionesParticipadas.to_pickle(dffileRelacionesParticipadas)
    dfwork.to_pickle(dffileNodosEmpresa)
    dfrelacionesCargos.to_pickle(dffileRelacionesCargos)
    onlyfiles1.remove(file)
    with open(counter_file_name, 'wb') as dumpPickleCounter:
        pickle.dump(onlyfiles1, dumpPickleCounter)

18:51:38  processing file:  ../../data/datasets/SABI/lista1/42 Lista1 Empresas 233702_239401 (1).xlsx
